In [3]:
import pandas as pd

import warnings
warnings.filterwarnings('ignore')

import missingno as msno
import seaborn as sns
from matplotlib import rc
import matplotlib.pyplot as plt
plt.rcParams["font.family"] = 'NanumGothic'
rc('font', weight='bold')
%matplotlib inline

### 시청률

In [4]:
rating_train = pd.read_excel('train/rating_train.xlsx', skiprows=1).iloc[:-1,:-1]
rating_train = pd.concat([pd.to_datetime(rating_train['시간대']), rating_train.iloc[:,1:]], axis = 1)
rating_train['hour'] = rating_train['시간대'].dt.hour
rating_train.loc[rating_train['hour']==0, 'hour'] = 24 
rating_train.head()

,시간대,2019-01-01,2019-01-02,2019-01-03,2019-01-04,2019-01-05,2019-01-06,2019-01-07,2019-01-08,2019-01-09,...,2019-12-23,2019-12-24,2019-12-25,2019-12-26,2019-12-27,2019-12-28,2019-12-29,2019-12-30,2019-12-31,hour
0,2020-08-24 02:00:00,0.0,0.0,0.000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000,0.0,0.0,0.0,0.0,0.0,0.000,2
1,2020-08-24 02:01:00,0.0,0.0,0.000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000,0.0,0.0,0.0,0.0,0.0,0.012,2
2,2020-08-24 02:02:00,0.0,0.0,0.000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000,0.0,0.0,0.0,0.0,0.0,0.000,2
3,2020-08-24 02:03:00,0.0,0.0,0.014,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.017,0.0,0.0,0.0,0.0,0.0,0.000,2
4,2020-08-24 02:04:00,0.0,0.0,0.000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.017,0.0,0.0,0.0,0.0,0.0,0.000,2


In [18]:
hday = [i.replace(' ','-') for i in [
'2019 01 01',
'2019 02 04',
'2019 02 05',
'2019 02 06',
'2019 03 01',
'2019 05 06',
'2019 06 06',
'2019 08 15',
'2019 09 12',
'2019 08 13',
'2019 10 03',
'2019 10 09',
'2019 12 25',
'2020 01 01',]]
t = ['mon', 'tue', 'wed', 'thur', 'fri', 'sat', 'sun'] # 요일
w_dic = {str(pd.to_datetime(k)).split()[0]:t[pd.to_datetime(k).weekday()] for k in rating_train.columns[1:-1]}
h_dic = {k: 1 if (w_dic[k]=='sun') or (w_dic[k]=='sat') or (k in hday) else 0 for k in w_dic.keys()}

In [72]:
# 월별(평일) 시간별 시청률
cols = [col for col in rating_train.columns[1:-1] if (h_dic[col]==0)] 
df = rating_train[cols]
df['hour'] = rating_train['hour']
df.index = rating_train['시간대'].astype('str').apply(lambda x: x.split()[1][:-3])
ht = df.pivot_table(index = 'hour', aggfunc = 'mean').T
ht = ht.reset_index()
ht['mth'] = ht['index'].apply(lambda x: int(x.split('-')[1]))
ht = ht.set_index(['index'])
print('\n*********************** 평일 월별 시간별 시청률 ***********************')
cm = sns.light_palette('blue', as_cmap=True)
display(ht.pivot_table(index = 'mth', aggfunc= 'mean').T.style.background_gradient(cmap=cm).format('{:.2%}'))

# 월별(휴일) 시간별 시청률
cols = [col for col in rating_train.columns[1:-1] if (h_dic[col]==1)] 
df = rating_train[cols]
df['hour'] = rating_train['hour']
df.index = rating_train['시간대'].astype('str').apply(lambda x: x.split()[1][:-3])
ht = df.pivot_table(index = 'hour', aggfunc = 'mean').T
ht = ht.reset_index()
ht['mth'] = ht['index'].apply(lambda x: int(x.split('-')[1]))
ht = ht.set_index(['index'])
print('\n*********************** 휴일 월별 시간별 시청률 ***********************')
display(ht.pivot_table(index = 'mth', aggfunc= 'mean').T.style.background_gradient(cmap=cm).format('{:.2%}'))


*********************** 평일 월별 시간별 시청률 ***********************


mth,1,2,3,4,5,6,7,8,9,10,11,12
hour,,,,,,,,,,,,
1,0.27%,0.57%,0.39%,0.23%,0.26%,0.21%,0.51%,0.45%,0.64%,0.64%,0.71%,0.96%
2,0.05%,0.29%,0.38%,0.25%,0.05%,0.29%,0.33%,0.42%,0.36%,0.38%,0.16%,0.58%
3,0.03%,0.19%,0.23%,0.22%,0.05%,0.24%,0.50%,0.29%,0.42%,0.29%,0.10%,0.33%
4,0.12%,0.09%,0.27%,0.11%,0.11%,0.25%,0.19%,0.17%,0.35%,0.06%,0.10%,0.12%
5,0.06%,0.09%,0.15%,0.24%,0.12%,0.15%,0.23%,0.11%,0.15%,0.14%,0.14%,0.12%
6,0.10%,0.10%,0.14%,0.18%,0.17%,0.15%,0.34%,0.13%,0.29%,0.30%,0.17%,0.10%
7,0.12%,0.17%,0.15%,0.11%,0.13%,0.27%,0.40%,0.32%,0.33%,0.28%,0.37%,0.31%
8,0.15%,0.27%,0.24%,0.24%,0.34%,0.33%,0.42%,0.60%,0.52%,0.35%,0.36%,0.40%
9,0.22%,0.23%,0.21%,0.39%,0.38%,0.31%,0.67%,0.71%,0.29%,0.38%,0.46%,0.76%



*********************** 휴일 월별 시간별 시청률 ***********************


mth,1,2,3,4,5,6,7,8,9,10,11,12
hour,,,,,,,,,,,,
1,0.10%,0.51%,0.51%,0.54%,0.28%,0.30%,0.11%,0.36%,0.34%,0.31%,0.18%,0.40%
2,0.38%,0.71%,0.12%,0.08%,0.17%,0.14%,0.32%,0.18%,0.19%,0.16%,0.32%,0.32%
3,0.11%,0.45%,0.05%,0.10%,0.08%,0.15%,0.30%,0.19%,0.07%,0.08%,0.33%,0.11%
4,0.15%,0.03%,0.16%,0.08%,0.29%,0.26%,0.55%,0.17%,0.10%,0.04%,0.44%,0.27%
5,0.11%,0.07%,0.19%,0.10%,0.09%,0.42%,0.06%,0.17%,0.26%,0.07%,0.61%,0.24%
6,0.11%,0.08%,0.09%,0.06%,0.05%,0.41%,0.10%,0.18%,0.20%,0.93%,0.65%,0.28%
7,0.16%,0.29%,0.14%,0.22%,0.33%,0.27%,0.41%,0.31%,0.35%,0.44%,0.74%,0.42%
8,0.22%,0.10%,0.67%,0.24%,0.54%,0.31%,0.67%,0.57%,0.41%,0.83%,0.53%,0.41%
9,0.13%,0.32%,0.54%,0.11%,0.18%,0.32%,0.37%,0.46%,0.69%,0.98%,0.24%,0.37%


In [ ]:
# 시청률과 매출액 관계

In [ ]:
# 2019 1 - 12 : ratings differ by Minutes (6시)
for mth in range(1, 12+1):
    cols = [col for col in rating_train.columns[1:-1] if pd.to_datetime(col).month==mth]
    df = rating_train[rating_train['hour']==6][cols]
    df.index = [i for i in range(60)]
    plt.figure(figsize = (20,10))
    sns.heatmap(df, annot=True)
    plt.xticks(rotation= 45)
    plt.xlabel('2019 - {}'.format(mth))
    plt.ylabel('minute')
    plt.title('2019 - {}, time: {}'.format(mth, 6))
    plt.show()